## Подготовка данных и раздведовательный анализ

In [24]:
# imports
import pandas as pd

#### Константы
- путь к данным
- признаки, описывающие состояние ТС

In [25]:
DATA_PATH = 'data/vehicle_dataset_public/'

TAGS = ['fuelLevel', 'ingection', 'refueling2',
        'speedAndHeight', 'tachometer']

In [29]:
def dict_df_vehicle_i(i):
    dict_df_vehiicle = {}
    for tag in TAGS:
        dict_df_vehiicle[tag] = pd.read_csv(DATA_PATH +
                                            f'vehicle{i}_{tag}_public.csv', sep=';')
    return dict_df_vehiicle

In [30]:
dict_df = dict_df_vehicle_i(1)

In [31]:
dict_df


{'fuelLevel':                      DTIME BEVALUE
 0      2020-01-08 21:16:02       0
 1      2020-01-08 21:17:04       0
 2      2020-01-08 21:18:04       0
 3      2020-01-09 10:05:26    49,7
 4      2020-01-09 10:06:27    49,9
 ...                    ...     ...
 19027  2020-06-27 00:48:05      54
 19028  2020-06-27 00:49:05    54,2
 19029  2020-06-27 01:15:14      54
 19030  2020-06-27 01:16:14    54,2
 19031  2020-06-27 01:17:14    54,4
 
 [19032 rows x 2 columns],
 'ingection':                      DTIME  BEVALUE
 0      2020-01-08 21:16:02        0
 1      2020-01-08 21:17:04        0
 2      2020-01-08 21:18:04        0
 3      2020-01-09 10:05:26        1
 4      2020-01-09 10:06:27        1
 ...                    ...      ...
 85923  2020-06-27 01:10:27        1
 85924  2020-06-27 01:15:14        1
 85925  2020-06-27 01:15:22        1
 85926  2020-06-27 01:16:14        1
 85927  2020-06-27 01:17:14        1
 
 [85928 rows x 2 columns],
 'refueling2':      TSID            STAR